![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [85]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [86]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!

In [87]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [88]:
# total bytes consumed
print(ds_jobs_transformed["gender"].nbytes)

153264


In [89]:
ds_jobs_transformed.columns

Index(['student_id', 'city', 'city_development_index', 'gender',
       'relevant_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours', 'job_change'],
      dtype='object')

In [90]:
# Displaying all the unique values and their frequency for each column:
columns_to_check = ["city", "gender", "relevant_experience", "enrolled_university", 
                      "education_level", "major_discipline", "experience", "company_size", "company_type", "last_new_job", "job_change"]

for col in columns_to_check:
    print(f"Value counts for column: {col}")
    print(ds_jobs_transformed[col].value_counts(dropna=False))
    print("-" * 50)

Value counts for column: city
city_103    4355
city_21     2702
city_16     1533
city_114    1336
city_160     845
            ... 
city_129       3
city_111       3
city_121       3
city_140       1
city_171       1
Name: city, Length: 123, dtype: int64
--------------------------------------------------
Value counts for column: gender
Male      13221
NaN        4508
Female     1238
Other       191
Name: gender, dtype: int64
--------------------------------------------------
Value counts for column: relevant_experience
Has relevant experience    13792
No relevant experience      5366
Name: relevant_experience, dtype: int64
--------------------------------------------------
Value counts for column: enrolled_university
no_enrollment       13817
Full time course     3757
Part time course     1198
NaN                   386
Name: enrolled_university, dtype: int64
--------------------------------------------------
Value counts for column: education_level
Graduate          11598
Masters      

### 1. Columns containing categories with only two factors must be stored as Booleans (bool)

In [91]:
ds_jobs_transformed["relevant_experience"] = ds_jobs_transformed["relevant_experience"].map({
    "Has relevant experience": True,
    "No relevant experience": False
}).astype(bool)
ds_jobs_transformed["relevant_experience"].value_counts()

True     13792
False     5366
Name: relevant_experience, dtype: int64

In [92]:
ds_jobs_transformed["job_change"] = ds_jobs_transformed["job_change"].map({
    1.0: True,
    0.0: False
}).astype(bool)
ds_jobs_transformed["job_change"].value_counts()

False    14381
True      4777
Name: job_change, dtype: int64

### 2. Columns containing integers only must be stored as 32-bit integers (int32)

In [93]:
ds_jobs_transformed["student_id"] = ds_jobs_transformed["student_id"].astype("int32")
ds_jobs_transformed["training_hours"] = ds_jobs_transformed["training_hours"].astype("int32")

print(ds_jobs_transformed[["student_id", "training_hours"]].dtypes)

student_id        int32
training_hours    int32
dtype: object


### 3. Columns containing floats must be stored as 16-bit floats (float16)

In [94]:
ds_jobs_transformed["city_development_index"] = ds_jobs_transformed["city_development_index"].astype("float16")

print(ds_jobs_transformed["city_development_index"].dtypes)

float16


### 4. Columns containing nominal categorical data must be stored as the category data type

In [95]:
columns_to_convert = ["city", "gender", "major_discipline", "company_type"]

ds_jobs_transformed[columns_to_convert] = ds_jobs_transformed[columns_to_convert].apply(lambda col: col.astype('category'))

print(ds_jobs_transformed.dtypes)

student_id                   int32
city                      category
city_development_index     float16
gender                    category
relevant_experience           bool
enrolled_university         object
education_level             object
major_discipline          category
experience                  object
company_size                object
company_type              category
last_new_job                object
training_hours               int32
job_change                    bool
dtype: object


### 5. Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column

In [96]:
# enrolled_university column
enrolled_university_order = ['no_enrollment', 'Part time course', 'Full time course']

ds_jobs_transformed['enrolled_university'] = pd.Categorical(
    ds_jobs_transformed['enrolled_university'], categories=enrolled_university_order, ordered=True
)

print(ds_jobs_transformed['enrolled_university'].dtype)

print(ds_jobs_transformed['enrolled_university'].value_counts(dropna=False))

category
no_enrollment       13817
Full time course     3757
Part time course     1198
NaN                   386
Name: enrolled_university, dtype: int64


In [97]:
# education_level column
education_level_order = ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd']

ds_jobs_transformed['education_level'] = pd.Categorical(
    ds_jobs_transformed['education_level'], categories=education_level_order, ordered=True
)

print(ds_jobs_transformed['education_level'].dtype)

print(ds_jobs_transformed['education_level'].value_counts(dropna=False))

category
Graduate          11598
Masters            4361
High School        2017
NaN                 460
Phd                 414
Primary School      308
Name: education_level, dtype: int64


In [98]:
# experience column
# experience_order = ['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20']
# experience_order = ['<1'] + [str(i) for i in range(1, 21)] + ['>20']
experience_order = ['<1'] + list(map(str, range(1, 21))) + ['>20']

ds_jobs_transformed['experience'] = pd.Categorical(
    ds_jobs_transformed['experience'], categories=experience_order, ordered=True
)

# ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].fillna('<1')

print(ds_jobs_transformed['experience'].dtype)

print(ds_jobs_transformed['experience'].value_counts(dropna=False))

category
>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
15      686
11      664
14      586
1       549
<1      522
16      508
12      494
13      399
17      342
19      304
18      280
20      148
NaN      65
Name: experience, dtype: int64


In [99]:
# company_size column

company_size_order = ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+']

ds_jobs_transformed['company_size'] = pd.Categorical(
    ds_jobs_transformed['company_size'], categories=company_size_order, ordered=True
)

print(ds_jobs_transformed['company_size'].dtype)

print(ds_jobs_transformed['company_size'].value_counts(dropna=False))

category
NaN          5938
50-99        3083
100-499      2571
10000+       2019
10-49        1471
1000-4999    1328
<10          1308
500-999       877
5000-9999     563
Name: company_size, dtype: int64


In [100]:
# last_new_job column
last_new_job_order = ['never', '1', '2', '3', '4', '>4']

ds_jobs_transformed['last_new_job'] = pd.Categorical(
    ds_jobs_transformed['last_new_job'], categories=last_new_job_order, ordered=True
)
#ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].fillna('-1')

print(ds_jobs_transformed['last_new_job'].dtype)

print(ds_jobs_transformed['last_new_job'].value_counts(dropna=False))

category
1        8040
>4       3290
2        2900
never    2452
4        1029
3        1024
NaN       423
Name: last_new_job, dtype: int64


In [101]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  bool    
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  bool    
dtypes: bool(2), category(9

In [102]:
ds_jobs.memory_usage()

Index                        128
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64

In [103]:
ds_jobs_transformed.memory_usage()

Index                       128
student_id                76632
city                      24310
city_development_index    38316
gender                    19290
relevant_experience       19158
enrolled_university       19290
education_level           19370
major_discipline          19378
experience                19890
company_size              19522
company_type              19378
last_new_job              19378
training_hours            76632
job_change                19158
dtype: int64

### 6. The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies

In [104]:
# Step 1: Filter students with 10 or more years of experience at companies with at least 1000 employees
ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed['experience'] >= '10') & (ds_jobs_transformed['company_size'] >= '1000-4999')]
ds_jobs_transformed.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,NaN,True,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,False
12,25619,city_61,0.913086,Male,True,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,False
31,22293,city_103,0.919922,Male,True,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,False
34,26494,city_16,0.910156,Male,True,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,False
40,2547,city_114,0.925781,Female,True,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,False
